In [1]:
import numpy as np
import pandas as pd
from bokeh.io import show, output_file
from bokeh.plotting import figure
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
terror = pd.read_csv("C:/Users/Reddy/Desktop/visual dataset/globalterrorismdb_0718dist.csv", encoding="ISO-8859-1")


In [5]:
terror.shape


(181691, 135)

check for the type for the data

check for the number fo missing values

In [6]:
terror.isnull().sum()


eventid            0
iyear              0
imonth             0
iday               0
approxdate    172452
               ...  
INT_LOG            0
INT_IDEO           0
INT_MISC           0
INT_ANY            0
related       156653
Length: 135, dtype: int64

In [4]:
terror.dtypes


eventid        int64
iyear          int64
imonth         int64
iday           int64
approxdate    object
               ...  
INT_LOG        int64
INT_IDEO       int64
INT_MISC       int64
INT_ANY        int64
related       object
Length: 135, dtype: object

In [ ]:
terror.isnull().sum()


renaming columns  

In [7]:
terror.rename(columns={'iyear':'Year','imonth':'Month','iday':'Day','country_txt':'Country','region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed','nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type','weaptype1_txt':'Weapon_type','motive':'Motive'},inplace=True)


In [8]:
terror=terror[['Year','Month','Day','Country','Region','city','latitude','longitude','AttackType','Killed','Wounded','Target','Summary','Group','Target_type','Weapon_type','Motive']]


creating casualties frame

In [9]:
terror['casualities']=terror['Killed']+terror['Wounded']


In [10]:
region = terror[['Region','Killed','Wounded','casualities']].copy()
region = region.groupby(by=['Region']).sum()
region

,Killed,Wounded,casualities
Region,,,
Australasia & Oceania,150.0,260.0,409.0
Central America & Caribbean,28708.0,8991.0,34530.0
Central Asia,1000.0,2009.0,2969.0
East Asia,1152.0,9213.0,10003.0
Eastern Europe,7415.0,12045.0,18239.0
Middle East & North Africa,137642.0,214308.0,329807.0
North America,4916.0,21531.0,26398.0
South America,28849.0,16704.0,43562.0
South Asia,101319.0,141360.0,234366.0


In [11]:
from bokeh.models import NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from math import pi
from bokeh.io import output_notebook
from bokeh.io import export_png
from bokeh.io import curdoc, output_notebook
from bokeh.models import HoverTool
#from bokeh.layouts import widgetbox, row, column
from bokeh.palettes import brewer
from bokeh.models import FixedTicker,NumeralTickFormatter


In [12]:
trend = terror.groupby(['Region','Year']).size().reset_index().rename(columns={0:'Killed'})

regions =trend.Region.unique()
regions

years= trend.Year.unique()

In [13]:
trend

,Region,Year,Killed
0,Australasia & Oceania,1970,1
1,Australasia & Oceania,1971,1
2,Australasia & Oceania,1972,8
3,Australasia & Oceania,1973,1
4,Australasia & Oceania,1974,1
...,...,...,...
524,Western Europe,2013,261
525,Western Europe,2014,215
526,Western Europe,2015,333
527,Western Europe,2016,273


In [14]:
import geopandas as gpd
shapefile ='C:/Users/Reddy/Desktop/shape/ne_110m_admin_0_countries.shp'
globalter = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
globalter.columns = ['Country', 'country_code', 'geometry']
#sorting dataframe
globalter.sort_values(by=['Country'], inplace=True)
#reseting index
globalter.reset_index(drop=True, inplace=True)

In [15]:
df= terror[['Country', 'Killed', 'Wounded' ]].groupby(by="Country").sum()
df.reset_index(inplace=True)
df['Casulties'] = df.Killed + df.Wounded
#sorting by country name
df.sort_values(by=['Country'],  inplace=True)
#reset index
df.reset_index(drop=True, inplace=True)

country names in shape file dataframe(globalter) and csv dataframe df must match to show countries on map choropleth


In [16]:
def get_countries_with_space(df):
    #returns -1 whnen character is not found
    #getting indexes of name containing spaces
    result_df= np.core.defchararray.find(df.astype(str), ' ')
    #print(result_df)
    indexes = np.where(result_df != -1)
    #print(indexes)
    return indexes,df[indexes]

In [17]:
df_countries= np.array(df.Country)

index_df, countries_df = get_countries_with_space(df_countries)


In [ ]:
changing names of countries 

In [18]:
replacement_df= ['Antigua and Barbuda', 'Burkina Faso', 'Central African Republic',
 'Costa Rica', 'Czechia', 'Republic of the Congo',
 'Dominican Republic', 'Germany' ,'East Timor' ,'El Salvador',
 'Equatorial Guinea' ,'Falkland Islands' ,'Guyana', 'French Polynesia',
 'Hong Kong', 'Ivory Coast', 'New Caledonia', 'New Hebrides', 'New Zealand',
 'North Korea', 'Yemen', 'Papua New Guinea',
 "Democratic Republic of the Congo", 'Republic of the Congo', 'Saudi Arabia',
 'Sierra Leone', 'Slovak Republic', 'Solomon Islands', 'South Africa',
 'South Korea', 'South Sudan', 'Vietnam', 'Yemen', 'Russia',
 'Sri Lanka', 'St. Kitts and Nevis', 'St. Lucia', 'Trinidad and Tobago',
 'United Arab Emirates', 'United Kingdom', 'United States of America', 'Vatican City',
 'Wallis and Futuna', 'West Bank and Gaza Strip', 'Germany','Western Sahara']


In [19]:
df_countries[index_df] = replacement_df

df.Country= df_countries

In [20]:
globalter_countries = np.array(globalter.Country)
df_countries = np.array(df.Country)

#getting nonnmatching values from globalter
nonmached_globalter=[]
for i in globalter_countries:
    if i not in df_countries:
        nonmached_globalter.append(i)

getting nonnmatching values from df


In [21]:
nonmached_df=[]
indexes = []
count=0
for i in df_countries:
    if i not in globalter_countries:
        nonmached_df.append(i)
        indexes.append(count)
    count = count +1

replacement= ['Andorra', 'Antigua and Barbuda', 'The Bahamas', 'Bahrain', 'Barbados', 'Bosnia and Herzegovina', 'Comoros', 'Czechia',
              'Dominica', 'French Polynesia', 'Grenada', 'Guadeloupe', 'Hong Kong', 'International', 'Macau', 'Maldives', 'Malta',
              'Martinique', 'Mauritius', 'New Hebrides', 'Rhodesia', 'Republic of Serbia', 'Republic of Serbia', 'Seychelles', 'Singapore',
              'Slovakia', 'St. Kitts and Nevis', 'St. Lucia','Swaziland', 'United Republic of Tanzania', 'Vatican City', 'Wallis and Futuna',
              'West Bank and Gaza Strip', 'Yugoslavia', 'Zaire']

In [ ]:
df_countries[indexes] = replacement
#replacing dataframe
df.Country = df_countries

summing the duplicate countries to merge

In [22]:
df= df.groupby(by='Country').sum()
df.reset_index(inplace= True)

merging the countries

In [23]:
merged_df = globalter.merge(df, left_on = 'Country', right_on = 'Country',  how = 'left')
merged_df= merged_df.sort_values(by='Casulties', ascending= False )
merged_df.dropna( inplace = True)


In [24]:
import json
#Read data to json.
merged_json = json.loads(merged_df.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

creating a choropleth

In [25]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.io import save
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

geosource = GeoJSONDataSource(geojson = json_data)

In [26]:
palette = brewer["YlOrRd"][8]
color_mapper = LinearColorMapper (palette = palette, low=0 ,high=max(merged_df.Casulties))
max_toll = merged_df.Casulties.max()
ticker = FixedTicker(ticks=[0,(max_toll*0.20),(max_toll*0.40),(max_toll*0.60),(max_toll*0.80),(max_toll)])

In [27]:
color_bar = ColorBar(color_mapper=color_mapper,width = 800, height = 20,ticker = ticker, formatter = NumeralTickFormatter(format=","),
border_line_color=None, orientation = 'horizontal', location=(0, 0) )

In [28]:
hover = HoverTool(tooltips = [ ('Country','@Country'),('KIlled', '@Killed')])
#Create figure object.
p = figure(title = 'Worldwide Casualties', plot_height = 600 , plot_width = 950,
           tools = [hover,"pan,wheel_zoom,box_zoom,reset" ],
           toolbar_location = "left")

In [29]:
#Add patch renderer to figure.
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Casulties', 'transform' : color_mapper},
          line_color = 'white', line_width = 1, fill_alpha = 0.7)
#Specify figure layout.
p.add_layout(color_bar, 'below')

show(p)


second map

In [30]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [32]:
df= terror.city.value_counts()
df.drop(axis=1, index='Unknown', inplace=True)
df =df.reset_index()
df.rename(columns={'index':'city','city':'attackcount'}, inplace= True)
df.sort_values(by=['attackcount'],ascending=False,  inplace= True)
top_target_cities = df.head(100)
top_target_cities

,city,attackcount
0,Baghdad,7589
1,Karachi,2652
2,Lima,2359
3,Mosul,2265
4,Belfast,2171
...,...,...
95,Batticaloa,165
96,Tegucigalpa,164
98,Berlin,160
97,Jamrud,160


In [33]:
index =0;

#get lat long
ter_city = terror[['latitude','longitude','city','Country']]
print(ter_city.shape)
ter_city.drop_duplicates(inplace= True)
print(ter_city.shape)

(181691, 4)
(56806, 4)


C:\Users\Reddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
merging the dataframe

In [34]:
top100 = top_target_cities.merge(ter_city, left_on = 'city', right_on = 'city',  how = 'left')
top100.drop_duplicates(subset=['city'],inplace=True)
top100.sort_values(by='attackcount', ascending= False )
top100.dropna( inplace = True)
top100.reset_index(inplace=True) #to reset index from 0
top100.drop(columns=['index'],inplace=True) #remove index deopped column
top100.dtypes

city            object
attackcount      int64
latitude       float64
longitude      float64
Country         object
dtype: object

In [35]:

from folium.plugins import MarkerCluster


m = folium.Map(location=[top100.iloc[0]['latitude'], top100.iloc[0]['longitude']], tiles="CartoDB dark_matter", zoom_start=3)

marker_cluster = MarkerCluster().add_to(m)

for i in range(0,top100.shape[0]):
    row = top100.iloc[i]
    folium.Circle(
      location=[row['latitude'], row['longitude']],
      popup= row['city']+', '+row['Country']+' \n Attack Counts: '+str(row['attackcount']),
      radius=(int(row['attackcount']))*50,
      color="#FFFFFF",
      fill=False,
      fill_color='white'
   ).add_to(m)
m

In [ ]:
Line chart

In [38]:

def set_stype(figure,  xlabel="", ylabel=""):
    #figure.title =
    figure.title.align ='center'

    figure.xaxis.axis_label=xlabel
    figure.yaxis.axis_label =ylabel
    figure.xaxis.axis_label_text_font="times"

    figure.yaxis.axis_label_text_font="times"
    figure.xaxis.axis_label_text_font_style ="bold"
    figure.yaxis.axis_label_text_font_style ="bold"

    figure.title.text_font = "times"
    figure.title.text_font_style = "bold"


In [ ]:
from bokeh.palettes import Cividis256
#from bokeh.charts import Area, show
df= terror.copy()
counts = terror.Year.value_counts()
counts.sort_index(ascending=False, inplace=True)

df = terror.groupby(by='Year' ).sum()
df.sort_index(ascending= False, inplace=True)
df['AttackCount'] = counts
df.reset_index(inplace=True)

cols = np.empty(shape=(df.Year.shape)  , dtype="object")
source = ColumnDataSource(df)

options = dict(plot_width=750, plot_height=500,
               tools=["pan,wheel_zoom,box_zoom,box_select,lasso_select", hover])


#hover1 = HoverTool(tooltips = [ ('Casualities','@casualities'), ('Year','@Year')])
p2 = figure(**options, title="Casualties by Year") #
p2.line( 'Year','casualities' ,color= 'red', alpha=0.8, line_width=4, source = source )
p2.circle(x='Year',y='casualities',  fill_color= 'Colors' , size=10, source = source, line_color = 'white')
p2.xaxis.major_label_orientation = pi/2
set_stype(p2,  xlabel="Year", ylabel= "Casualties" )

show(p2)